<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">  

<b> <font size='5'>  
ELEC PRICE PREDICTION CAPSTONE:</font>  

<font size='4'>CAISO OASIS API Data Acquisition Notebook</font> </b>

<font size='3'>  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Manu Kalia Project Submission<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; DSI-7-SF<br>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 15-May-2019<br>
</font>

---


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-modules-and-classes" data-toc-modified-id="Import-modules-and-classes-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import modules and classes</a></span></li><li><span><a href="#Functions-to-Construct-Queries" data-toc-modified-id="Functions-to-Construct-Queries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions to Construct Queries</a></span></li><li><span><a href="#Get-Energy-Price-Zipfiles" data-toc-modified-id="Get-Energy-Price-Zipfiles-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get Energy Price Zipfiles</a></span></li><li><span><a href="#Unzip-Downloaded-CAISO-Files" data-toc-modified-id="Unzip-Downloaded-CAISO-Files-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Unzip Downloaded CAISO Files</a></span></li></ul></div>

---

## Import modules and classes

In [1]:
import wget, os, zipfile
import time

---

## Functions to Construct Queries

(for Accessing CAISO OASIS API)

**CSV download query format... DAM**  

http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=1&startdatetime=20160101T08:00-0000&enddatetime=20160201T08:00-0000&market_run_id=DAM&node=BAYSHOR2_1_N001

**CSV download query format... HASP**  

http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_HASP_LMP&version=3&startdatetime=20160101T08:00-0000&enddatetime=20160202T08:00-0000&market_run_id=HASP&node=BAYSHOR2_1_N001

**CSV download query format... RTM**

http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_INTVL_LMP&version=3&startdatetime=20190505T07:00-0000&enddatetime=20190505T08:00-0000&market_run_id=RTM&node=BAYSHOR2_1_N001

**CSV download query format... 7 Day Ahead Load F'cast**

http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&market_run_id=7DA&startdatetime=20190505T07:00-0000&enddatetime=20190506T07:00-0000

**CSV download query format... 2 Day Ahead Load F'cast**

http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&market_run_id=2DA&startdatetime=20190505T07:00-0000&enddatetime=20190506T07:00-0000

**CSV download query format... DAM Load F'cast**

http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&market_run_id=DAM&startdatetime=20190505T07:00-0000&enddatetime=20190506T07:00-0000

**CSV download query format... RTM Load F'cast**

http://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=SLD_FCST&version=1&market_run_id=RTM&startdatetime=20190505T07:00-0000&enddatetime=20190506T07:00-0000

<br/> 

**Mandatory Parameters**

|Parameter    |Description                                                         |
|-------------|--------------------------------------------------------------------|
|startdatetime|valid operating start datetime in GMT (yyyymmddThh24:miZ)           |
|enddatetime  |valid operating end datetime in GMT (yyyymmddThh24:miZ)             |
|resultformat |'6' for .csv file download                                          |
|queryname    |'PRC_LMP' for DAM LMP prices, 'PRC_HASP_LMP' for HASP LMP prices    |
|market_run_id|'DAM' for Day-Ahead-Market, 'HASP' for Hour-Ahead-Scheduling Process|
|version      |API version ('1' for the DAM query, '3' for HASP query)             |
|node         |Use the San Francisco Bay Shore node site as the example node       |


In [2]:
# Function to construct a one-month electricity Day-Ahead-Market
#    data query for the CAISO OASIS API

# Parameters (all strings):
#    node                       name of node
#    startyear, startmonth


def create_DAM_query(node, startyear, startmonth):
    oasis_website = 'oasis.caiso.com'
    context_path  = 'oasisapi'

    url = f'http://{oasis_website}/{context_path}/SingleZip'

    resultformat  =  '6'
    queryname     =  'PRC_LMP'
    version       =  '1'
    market_run_id =  'DAM'
    
    if startmonth == '12':
        endmonth  =  '01'
        endyear   =  f'{int(startyear) + 1}'
    else:
        endmonth  =  f'{(int(startmonth) + 1):02d}'
        endyear   =  startyear

    startdatetime =  f'{startyear}{startmonth}01T08:00-0000'
    enddatetime   =  f'{endyear}{endmonth}01T07:00-0000'

    query = f'{url}?resultformat={resultformat}&queryname={queryname}\
&version={version}&startdatetime={startdatetime}&enddatetime={enddatetime}\
&market_run_id={market_run_id}&node={node}'
    
    return query

In [3]:
# Function to construct a one-month electricity Hour-Ahead-Scheduling-Process
#    data query for the CAISO OASIS API

# Parameters (all strings):
#    node                       name of node
#    startyear, startmonth


def create_HASP_query(node, startyear, startmonth):
    oasis_website = 'oasis.caiso.com'
    context_path  = 'oasisapi'

    url = f'http://{oasis_website}/{context_path}/SingleZip'

    resultformat  =  '6'
    queryname     =  'PRC_HASP_LMP'
    version       =  '3'
    market_run_id =  'HASP'
    
    if startmonth == '12':
        endmonth  =  '01'
        endyear   =  f'{int(startyear) + 1}'
    else:
        endmonth  =  f'{(int(startmonth) + 1):02d}'
        endyear   =  startyear

    startdatetime =  f'{startyear}{startmonth}01T08:00-0000'
    enddatetime   =  f'{endyear}{endmonth}01T07:00-0000'

    query = f'{url}?resultformat={resultformat}&queryname={queryname}\
&version={version}&startdatetime={startdatetime}&enddatetime={enddatetime}\
&market_run_id={market_run_id}&node={node}'
    
    return query

In [4]:
# Function to construct a GENERIC one-month .csv LMP price download
#    data query for the CAISO OASIS API

# Parameters (all strings):
#    node                       name of node
#    startyear, startmonth


def price_query(queryname, version, startyear, startmonth, market_run_id, node):
    oasis_website = 'oasis.caiso.com'
    context_path  = 'oasisapi'

    url = f'http://{oasis_website}/{context_path}/SingleZip'

    resultformat  =  '6'
    
    if startmonth == '12':
        endmonth  =  '01'
        endyear   =  f'{int(startyear) + 1}'
    else:
        endmonth  =  f'{(int(startmonth) + 1):02d}'
        endyear   =  startyear

    startdatetime =  f'{startyear}{startmonth}01T08:00-0000'
    enddatetime   =  f'{endyear}{endmonth}01T07:00-0000'

    query = f'{url}?resultformat={resultformat}&queryname={queryname}&version={version}\
&startdatetime={startdatetime}&enddatetime={enddatetime}\
&market_run_id={market_run_id}&node={node}'
    
    return query

In [11]:
# Function to construct a GENERIC one-month .csv load (demand) forecast
#    download data query for the CAISO OASIS API

# Parameters (all strings):
#    startyear, startmonth


def load_query(queryname, version, startyear, startmonth, market_run_id):
    oasis_website = 'oasis.caiso.com'
    context_path  = 'oasisapi'

    url = f'http://{oasis_website}/{context_path}/SingleZip'

    resultformat  =  '6'
    
    if startmonth == '12':
        endmonth  =  '01'
        endyear   =  f'{int(startyear) + 1}'
    else:
        endmonth  =  f'{(int(startmonth) + 1):02d}'
        endyear   =  startyear

    startdatetime =  f'{startyear}{startmonth}01T08:00-0000'
    enddatetime   =  f'{endyear}{endmonth}01T07:00-0000'

    query = f'{url}?resultformat={resultformat}&queryname={queryname}&version={version}\
&market_run_id={market_run_id}&startdatetime={startdatetime}&enddatetime={enddatetime}'
    
    return query

---

## Get Energy Price Zipfiles 

(from CAISO OASIS website via API)

In [6]:
# node = 'BAYSHOR2_1_N001'

# for i in range(1, 41):
#     if i % 12 == 0:
#         startyear  = str(2016 + i//12 - 1)
#         startmonth = f'{12:02d}'
#     else:
#         startyear  = str(2016 + i//12)
#         startmonth = f'{i%12:02d}'
    
#     wget.download(price_query('PRC_INTVL_LMP',
#                               '3',
#                               startyear,
#                               startmonth,
#                               'RTM',
#                               node), '../caiso_rtm_dl/')
#     time.sleep(5)

In [13]:
for i in range(1, 41):
    if i % 12 == 0:
        startyear  = str(2016 + i//12 - 1)
        startmonth = f'{12:02d}'
    else:
        startyear  = str(2016 + i//12)
        startmonth = f'{i%12:02d}'
    
    wget.download(load_query('SLD_FCST',
                             '1',
                             startyear,
                             startmonth,
                             '7DA',
                            ), '../caiso_7da_load_dl/')
    time.sleep(5)    
    
    wget.download(load_query('SLD_FCST',
                             '1',
                             startyear,
                             startmonth,
                             '2DA',
                            ), '../caiso_2da_load_dl/')
    time.sleep(5)    
    
    
    wget.download(load_query('SLD_FCST',
                             '1',
                             startyear,
                             startmonth,
                             'DAM',
                            ), '../caiso_dam_load_dl/')
    time.sleep(5)    
    
    wget.download(load_query('SLD_FCST',
                             '1',
                             startyear,
                             startmonth,
                             'RTM',
                            ), '../caiso_rtm_load_dl/')
    time.sleep(5)    

In [27]:
# Single month download cell

# caiso_dam_dl = '../caiso_dam_dl/'
# caiso_hasp_dl = '../caiso_hasp_dl/'

# node = 'BAYSHOR2_1_N001'

# startyear  = '2016'
# startmonth = '05'

# wget.download(create_DAM_query(node, startyear, startmonth), caiso_dam_dl)
# time.sleep(10)
# wget.download(create_HASP_query(node, startyear, startmonth), caiso_hasp_dl)

---

## Unzip Downloaded CAISO Files

(to .csv Directories)

In [4]:
unzipped_caiso_dam = '../unzipped_caiso_dam/'
unzipped_caiso_hasp = '../unzipped_caiso_hasp/'

caiso_dam_dl = '../caiso_dam_dl/'
caiso_hasp_dl = '../caiso_hasp_dl/'

In [5]:
for item in os.listdir(caiso_dam_dl):           # loop through items in dir
    if item.split('.')[-1] == 'zip':            # check for zip extension
        file_name = f'{caiso_dam_dl}{item}'     # get relative path of files
        print(f'unzipping... {file_name}')
        zip_ref = zipfile.ZipFile(file_name)    # create zipfile object
        with zip_ref as target:
            target.extractall(unzipped_caiso_dam)
    else: continue

unzipping... ../caiso_dam_dl/20170701_20170801_PRC_LMP_DAM_20190501_02_57_46_v1.zip
unzipping... ../caiso_dam_dl/20160501_20160601_PRC_LMP_DAM_20190501_02_27_13_v1.zip
unzipping... ../caiso_dam_dl/20180701_20180801_PRC_LMP_DAM_20190501_03_21_17_v1.zip
unzipping... ../caiso_dam_dl/20160701_20160801_PRC_LMP_DAM_20190501_02_32_47_v1.zip
unzipping... ../caiso_dam_dl/20160101_20160201_PRC_LMP_DAM_20190501_01_06_29_v1.zip
unzipping... ../caiso_dam_dl/20160601_20160701_PRC_LMP_DAM_20190501_01_23_53_v1.zip
unzipping... ../caiso_dam_dl/20190201_20190228_PRC_LMP_DAM_20190501_03_28_55_v1.zip
unzipping... ../caiso_dam_dl/20180201_20180228_PRC_LMP_DAM_20190501_03_12_02_v1.zip
unzipping... ../caiso_dam_dl/20160301_20160401_PRC_LMP_DAM_20190501_01_08_36_v1.zip
unzipping... ../caiso_dam_dl/20161101_20161130_PRC_LMP_DAM_20190501_02_40_20_v1.zip
unzipping... ../caiso_dam_dl/20170401_20170501_PRC_LMP_DAM_20190501_02_51_08_v1.zip
unzipping... ../caiso_dam_dl/20161201_20161231_PRC_LMP_DAM_20190501_02_42_27

In [6]:
for item in os.listdir(caiso_hasp_dl):           # loop through items in dir
    if item.split('.')[-1] == 'zip':            # check for zip extension
        file_name = f'{caiso_hasp_dl}{item}'     # get relative path of files
        print(f'unzipping... {file_name}')
        zip_ref = zipfile.ZipFile(file_name)    # create zipfile object
        with zip_ref as target:
            target.extractall(unzipped_caiso_hasp)
    else: continue

unzipping... ../caiso_hasp_dl/20160401_20160501_PRC_HASP_LMP_HASP_20190501_01_10_20_v3.zip
unzipping... ../caiso_hasp_dl/20171001_20171101_PRC_HASP_LMP_HASP_20190501_03_04_55_v3.zip
unzipping... ../caiso_hasp_dl/20180201_20180228_PRC_HASP_LMP_HASP_20190501_03_12_30_v3.zip
unzipping... ../caiso_hasp_dl/20190101_20190131_PRC_HASP_LMP_HASP_20190501_03_28_34_v3.zip
unzipping... ../caiso_hasp_dl/20160301_20160401_PRC_HASP_LMP_HASP_20190501_01_08_51_v3.zip
unzipping... ../caiso_hasp_dl/20170701_20170801_PRC_HASP_LMP_HASP_20190501_02_58_18_v3.zip
unzipping... ../caiso_hasp_dl/20161201_20161231_PRC_HASP_LMP_HASP_20190501_02_43_08_v3.zip
unzipping... ../caiso_hasp_dl/20190201_20190228_PRC_HASP_LMP_HASP_20190501_03_29_09_v3.zip
unzipping... ../caiso_hasp_dl/20181101_20181130_PRC_HASP_LMP_HASP_20190501_03_27_23_v3.zip
unzipping... ../caiso_hasp_dl/20171201_20171231_PRC_HASP_LMP_HASP_20190501_03_09_09_v3.zip
unzipping... ../caiso_hasp_dl/20160101_20160201_PRC_HASP_LMP_HASP_20190501_01_06_47_v3.zip

In [8]:
download_dir = '../caiso_rtm_dl/'
unzipped_target_dir = '../unzipped_caiso_rtm/'

for item in os.listdir(download_dir):           # loop through items in dir
    if item.split('.')[-1] == 'zip':            # check for zip extension
        file_name = f'{download_dir}{item}'     # get relative path of files
        print(f'unzipping... {file_name}')
        zip_ref = zipfile.ZipFile(file_name)    # create zipfile object
        with zip_ref as target:
            target.extractall(unzipped_target_dir)
    else: continue

unzipping... ../caiso_rtm_dl/20180601_20180701_PRC_INTVL_LMP_RTM_20190506_01_50_22_v3.zip
unzipping... ../caiso_rtm_dl/20160401_20160501_PRC_INTVL_LMP_RTM_20190506_00_16_32_v3.zip
unzipping... ../caiso_rtm_dl/20170801_20170901_PRC_INTVL_LMP_RTM_20190506_01_19_02_v3.zip
unzipping... ../caiso_rtm_dl/20190401_20190501_PRC_INTVL_LMP_RTM_20190506_02_07_21_v3.zip
unzipping... ../caiso_rtm_dl/20190301_20190401_PRC_INTVL_LMP_RTM_20190506_02_06_40_v3.zip
unzipping... ../caiso_rtm_dl/20190201_20190228_PRC_INTVL_LMP_RTM_20190506_02_06_02_v3.zip
unzipping... ../caiso_rtm_dl/20171101_20171130_PRC_INTVL_LMP_RTM_20190506_01_28_49_v3.zip
unzipping... ../caiso_rtm_dl/20170901_20171001_PRC_INTVL_LMP_RTM_20190506_01_22_05_v3.zip
unzipping... ../caiso_rtm_dl/20180701_20180801_PRC_INTVL_LMP_RTM_20190506_01_53_55_v3.zip
unzipping... ../caiso_rtm_dl/20170501_20170601_PRC_INTVL_LMP_RTM_20190506_01_06_42_v3.zip
unzipping... ../caiso_rtm_dl/20160801_20160901_PRC_INTVL_LMP_RTM_20190506_00_30_03_v3.zip
unzipping.

In [15]:
def unzip_dir(download_dir, unzipped_target_dir):
    for item in os.listdir(download_dir):           # loop through items in dir
        if item.split('.')[-1] == 'zip':            # check for zip extension
            file_name = f'{download_dir}{item}'     # get relative path of files
            print(f'unzipping... {file_name}')
            zip_ref = zipfile.ZipFile(file_name)    # create zipfile object
            with zip_ref as target:
                target.extractall(unzipped_target_dir)
        else: continue

In [16]:
unzip_dir('../caiso_7da_load_dl/', '../unzipped_caiso_7da_load/')

unzipping... ../caiso_7da_load_dl/20190401_20190501_SLD_FCST_7DA_20190506_02_50_24_v1.zip
unzipping... ../caiso_7da_load_dl/20160501_20160601_SLD_FCST_7DA_20190506_02_33_00_v1.zip
unzipping... ../caiso_7da_load_dl/20170501_20170601_SLD_FCST_7DA_20190506_02_38_39_v1.zip
unzipping... ../caiso_7da_load_dl/20170901_20171001_SLD_FCST_7DA_20190506_02_40_36_v1.zip
unzipping... ../caiso_7da_load_dl/20160301_20160401_SLD_FCST_7DA_20190506_02_31_58_v1.zip
unzipping... ../caiso_7da_load_dl/20160701_20160801_SLD_FCST_7DA_20190506_02_33_55_v1.zip
unzipping... ../caiso_7da_load_dl/20160601_20160701_SLD_FCST_7DA_20190506_02_33_27_v1.zip
unzipping... ../caiso_7da_load_dl/20171201_20171231_SLD_FCST_7DA_20190506_02_42_09_v1.zip
unzipping... ../caiso_7da_load_dl/20180601_20180701_SLD_FCST_7DA_20190506_02_45_07_v1.zip
unzipping... ../caiso_7da_load_dl/20171101_20171130_SLD_FCST_7DA_20190506_02_41_37_v1.zip
unzipping... ../caiso_7da_load_dl/20170401_20170501_SLD_FCST_7DA_20190506_02_38_10_v1.zip
unzipping.

In [17]:
unzip_dir('../caiso_2da_load_dl/', '../unzipped_caiso_2da_load/')

unzipping... ../caiso_2da_load_dl/20170601_20170701_SLD_FCST_2DA_20190506_02_39_13_v1.zip
unzipping... ../caiso_2da_load_dl/20180701_20180801_SLD_FCST_2DA_20190506_02_45_46_v1.zip
unzipping... ../caiso_2da_load_dl/20181101_20181130_SLD_FCST_2DA_20190506_02_47_52_v1.zip
unzipping... ../caiso_2da_load_dl/20160901_20161001_SLD_FCST_2DA_20190506_02_34_56_v1.zip
unzipping... ../caiso_2da_load_dl/20160701_20160801_SLD_FCST_2DA_20190506_02_34_01_v1.zip
unzipping... ../caiso_2da_load_dl/20170801_20170901_SLD_FCST_2DA_20190506_02_40_13_v1.zip
unzipping... ../caiso_2da_load_dl/20170201_20170228_SLD_FCST_2DA_20190506_02_37_18_v1.zip
unzipping... ../caiso_2da_load_dl/20181201_20181231_SLD_FCST_2DA_20190506_02_48_23_v1.zip
unzipping... ../caiso_2da_load_dl/20181001_20181101_SLD_FCST_2DA_20190506_02_47_20_v1.zip
unzipping... ../caiso_2da_load_dl/20180501_20180601_SLD_FCST_2DA_20190506_02_44_42_v1.zip
unzipping... ../caiso_2da_load_dl/20180801_20180901_SLD_FCST_2DA_20190506_02_46_18_v1.zip
unzipping.

In [18]:
unzip_dir('../caiso_dam_load_dl/', '../unzipped_caiso_dam_load/')

unzipping... ../caiso_dam_load_dl/20170901_20171001_SLD_FCST_DAM_20190506_02_40_47_v1.zip
unzipping... ../caiso_dam_load_dl/20180901_20181001_SLD_FCST_DAM_20190506_02_46_54_v1.zip
unzipping... ../caiso_dam_load_dl/20160401_20160501_SLD_FCST_DAM_20190506_02_32_42_v1.zip
unzipping... ../caiso_dam_load_dl/20170601_20170701_SLD_FCST_DAM_20190506_02_39_20_v1.zip
unzipping... ../caiso_dam_load_dl/20180601_20180701_SLD_FCST_DAM_20190506_02_45_21_v1.zip
unzipping... ../caiso_dam_load_dl/20190301_20190401_SLD_FCST_DAM_20190506_02_50_03_v1.zip
unzipping... ../caiso_dam_load_dl/20161201_20161231_SLD_FCST_DAM_20190506_02_36_25_v1.zip
unzipping... ../caiso_dam_load_dl/20170701_20170801_SLD_FCST_DAM_20190506_02_39_50_v1.zip
unzipping... ../caiso_dam_load_dl/20160801_20160901_SLD_FCST_DAM_20190506_02_34_34_v1.zip
unzipping... ../caiso_dam_load_dl/20160501_20160601_SLD_FCST_DAM_20190506_02_33_12_v1.zip
unzipping... ../caiso_dam_load_dl/20160701_20160801_SLD_FCST_DAM_20190506_02_34_06_v1.zip
unzipping.

In [19]:
unzip_dir('../caiso_rtm_load_dl/', '../unzipped_caiso_rtm_load/')

unzipping... ../caiso_rtm_load_dl/20160601_20160701_SLD_FCST_RTM_20190506_02_33_45_v1.zip
unzipping... ../caiso_rtm_load_dl/20181201_20181231_SLD_FCST_RTM_20190506_02_48_35_v1.zip
unzipping... ../caiso_rtm_load_dl/20170401_20170501_SLD_FCST_RTM_20190506_02_38_27_v1.zip
unzipping... ../caiso_rtm_load_dl/20180501_20180601_SLD_FCST_RTM_20190506_02_44_54_v1.zip
unzipping... ../caiso_rtm_load_dl/20190201_20190228_SLD_FCST_RTM_20190506_02_49_37_v1.zip
unzipping... ../caiso_rtm_load_dl/20161101_20161130_SLD_FCST_RTM_20190506_02_36_03_v1.zip
unzipping... ../caiso_rtm_load_dl/20190101_20190131_SLD_FCST_RTM_20190506_02_49_06_v1.zip
unzipping... ../caiso_rtm_load_dl/20160801_20160901_SLD_FCST_RTM_20190506_02_34_40_v1.zip
unzipping... ../caiso_rtm_load_dl/20190401_20190501_SLD_FCST_RTM_20190506_02_50_42_v1.zip
unzipping... ../caiso_rtm_load_dl/20180701_20180801_SLD_FCST_RTM_20190506_02_45_58_v1.zip
unzipping... ../caiso_rtm_load_dl/20160501_20160601_SLD_FCST_RTM_20190506_02_33_17_v1.zip
unzipping.